In [3]:
# %rm -rf phase2_module
# !git clone https://github.com/LgDNet/phase2_module.git
# !cd phase2_module && make copy

In [1]:
cd ..

/home/elicer


/home/elicer/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [16]:
from phase2_module.src.utils.dataset import Data
from phase2_module.src.models.classification.models import Model
from phase2_module.src.base import BasePiepline
# from phase2_module.src.hyper_parameters.params_optimization import xgboost_opt, set_params_optimization_data
from phase2_module.src.utils.set_seed import seed_everything
from phase2_module.src.utils.top_score_instance import check_the_score
from phase2_module.src.encoder.encoders import Encode

from phase2_module.src.utils.manage_pkl_files import PickleManager as pkl_manager
from phase2_module.src import preprocess
seed_everything()  # NOTE: default 42
_model = Model()

In [17]:
import pandas as pd
import numpy as np
import re, pickle
import warnings
from tqdm import tqdm
warnings.filterwarnings(action='ignore')
def set_up(mode= None):
    pd.set_option('display.max_columns', mode)
set_up()

# Preprocessing

In [18]:
basic = preprocess.Basic
country = preprocess.Country
customer = preprocess.Customer
inquiry = preprocess.Inquiry
product_category = preprocess.ProductCategory
convert_ratio = preprocess.Convert_ratio
customer2 = preprocess.Customer2
category = preprocess.Categorization

# Main

In [19]:
pkls = pkl_manager.map(pkl_manager.loads, pkl_manager.metadata_directory)

runtime_instance = {
    "Train": Data.train,
    "Test": Data.test,
    "Instances":[
        {"instance": basic, "method": ['drop_duplicated'], "params": False},
        {"instance": country, "method": ['apply'], "params": pkls.get("country")},
        {"instance": inquiry, "method": ["apply"], "params": False},
        {"instance": product_category, "method": ["product_categories", 'fill_odds'], "params": {**pkls.get("product_category"), **pkls.get("product_subcategory")}},
        {"instance": customer, "method": ["apply"], "params": pkls.get("customer")},
        {"instance": category, "method": ["cat_customer_idx",'historical_cnt_mean'], "params": False},
        {"instance": convert_ratio, "method": ["lead_owner"], "params": {**pkls.get("lead_owner"), **pkls.get("customer_idx")}},
        {"instance": basic, "method": ['customer_idx_merge_enterprise'], "params": False},
        {"instance": customer2, "method": ['seniority_level', 'job_function', 'customer_type'], "params": pkls.get("customer2")},
#         {"instance": category, "method": ['customer_idx_merge_enterprise'], "params": False},
    ]
}

## preprocessing

In [20]:
df_train = preprocess.runner('Train',**runtime_instance)
df_test = preprocess.runner('Test',**runtime_instance)

In [105]:
df_train = all_train.copy()
df_test = all_test.copy()

In [7]:
cond = df_train['new_inquiry_type'] == 'Quotation or Purchase Consultation'
df_train.loc[cond, "less_timeline"] += 1
cond = df_test['new_inquiry_type'] == 'Quotation or Purchase Consultation'
df_test.loc[cond, "less_timeline"] += 1

cond = df_train['is_converted'] == True
true_num = df_train[cond]['lead_desc_length'].mean()
cond1 = df_train['lead_desc_length'] > true_num
df_train.loc[cond1,'less_timeline'] += 1
cond2 = df_test['lead_desc_length'] > true_num
df_test.loc[cond2,'less_timeline'] += 1

## encoding

In [8]:
encode = Encode()

In [9]:
exisiting_columns = [
                     'business_area','business_subarea','business_unit',
                        'enterprise',
#                         'response_corporate',#"customer_country", # country
#                         'customer_type','customer_job','customer_position', #customer
#                         'inquiry_type','expected_timeline', # inquiry
#                         'product_subcategory','product_category','product_modelname' # product_category
                       ] 
preprocess_columns = [
                     'country','continent', # country
                     'customer_type2','job_function','seniority_level', # customer
                     'new_expected_timeline', 'new_inquiry_type',# inquiry
                     'category_2','category_3','category_1',
                     'new_customer_type1','new_customer_type2' # product_category
#                      'test2'
                     ]

label_encode_columns = exisiting_columns + preprocess_columns

In [10]:
# 결측치 채우기
df_train = preprocess.fillna(df_train)
df_test = preprocess.fillna(df_test)

In [11]:
# # one-hot encoding
# onehot_encode_columns = ['category_1']
# df_train = encode.onehot_encoder(df_train,onehot_encode_columns)
# df_test = encode.onehot_encoder(df_test,onehot_encode_columns)

In [12]:
# label encoding
encode.set_up() # 전역 변수 초기화
df_train = encode.label_encoder(df_train,label_encode_columns)
df_test = encode.label_encoder(df_test,label_encode_columns)

# Modeling

## drop column
모델링 전이 column을 drop하면서 다루기가 쉽다고 판단하였음

In [13]:
drop_col = [
            'id_strategic_ver','it_strategic_ver','idit_strategic_ver','ver_pro','ver_cus',
            'customer_country.1',
            "customer_country",#'response_corporate' # country
            'customer_type','customer_job','customer_position', # customer
            'inquiry_type', 'expected_timeline', #inquiry
            'product_subcategory','product_category','product_modelname', # product_category
#             'cate_is_nan', # product_category
    
#             'com_reg_ver_win_rate','idit_strategic_ver','ver_cus',
#             'ver_pro','ver_win_rate_x','ver_win_ratio_per_bu',
            'customer_idx',
            'lead_owner','historical_existing_mean','response_corporate',#'new_customer_idx'
            'customer_type2',
#             'customer_idx_converted_ratio', # ratio
            #'lead_owner', 'lead_owner_converted_ratio', 'lead_owner_customer_idx_mean', # lead_owner
            ]
df_train.drop(drop_col,axis = 1, inplace = True)
df_test.drop(drop_col,axis = 1, inplace = True)

## Scaler

In [14]:
import numpy as np
# log 변환
df_train['lead_desc_length'] = np.log1p(df_train['lead_desc_length'])
df_test['lead_desc_length'] = np.log1p(df_test['lead_desc_length'])

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_train[['lead_owner_converted_ratio']] = scaler.fit_transform(df_train[['lead_owner_converted_ratio']])
df_test[['lead_owner_converted_ratio']] = scaler.transform(df_test[['lead_owner_converted_ratio']])

## type

In [16]:
df_train['new_customer_idx'] = df_train['new_customer_idx'].astype(int)
df_test['new_customer_idx'] = df_test['new_customer_idx'].astype(int)

## train

#### param

In [27]:
import math
from six.moves import xrange

class LoglossObjective(object):
    def calc_ders_range(self, approxes, targets, weights):
        assert len(approxes) == len(targets)
        if weights is not None:
            assert len(weights) == len(approxes)
        
        result = []
        for index in range(len(targets)):
            e = np.exp(approxes[index])
            p = e / (1 + e)
            der1 = targets[index] - p
            der2 = -p * (1 - p)
            

            if (targets[index] == 0.0) & (p<0.5):
                #( True Negative )
                der1 = (p)*(targets[index] - p)
                der2 = (1-2*p)*(-p * (1 - p))
                
            elif (targets[index] == 0.0) & (p>=0.5):
                # 타겟값이 노말 유저인데,fraud 라고 예측한 경우 ( False Positive )
                der1 = 3*(p)*(targets[index] - p)
                der2 = 3*(1-2*p)*(-p * (1 - p))

            elif (targets[index] > 0.0) :
                # 타겟 값이 fraud 이라면 p값이 높을수록 (정답에 가까울수록) 페널티가 적도록 세팅. 추가로 3배 페널티
                der1 = 3*(1-p)*(targets[index] - p)
                der2 = 3*(2*p-1)*(-p * (1 - p))
                
            if weights is not None:
                der1 *= weights[index]
                der2 *= weights[index]

            result.append((der1, der2))
        return result

In [28]:
cat_params = {'iterations':5000, 'depth':12, 'thread_count':3, 'learning_rate':1e-3, 'loss_function':LoglossObjective(), 'eval_metric':'PRAUC', 'early_stopping_rounds':400, 'verbose':1000}
decision_params = {'criterion': 'log_loss', 'max_depth': 20, 'min_samples_split': 7, 'min_samples_leaf': 2, 'random_state': 42}
xgb_params = {'n_estimators': 2393, 'max_depth': 16, 'min_child_weight': 2, 'learning_rate': 0.06783336462585174, 'subsample': 0.9511145375386896, 'gamma': 0.16846211770550093, 'colsample_bytree': 0.6662896237544925, 'colsample_bylevel': 0.8781042717330898, 'colsample_bynode': 0.741271661588135, 'random_state':42}
lgb_params = {'num_iterations' : 5000, 'objective' : 'binary', 'is_unbalance':True, 'max_depth': -1, 'early_stopping_rounds' : 500, 'verbose':0, 'seed':42}
random_params = {'max_depth': 19, 'max_leaf_nodes': 749, 'n_estimators': 349,'random_state':42}

### catboost

In [29]:
model = _model.catboost
model.set_params(**cat_params)

### decision_tree

In [30]:
model = _model.decision_tree
model.set_params(**decision_params)

DecisionTreeClassifier(criterion='log_loss', max_depth=20, min_samples_leaf=2,
                       min_samples_split=7, random_state=42)

### xgboost

In [31]:
model = _model.xgboost
model.set_params(**xgb_params)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.8781042717330898,
              colsample_bynode=0.741271661588135,
              colsample_bytree=0.6662896237544925, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.16846211770550093,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.06783336462585174,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=16, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2393, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

### light_gbm

In [32]:
model = _model.lightgbm
model.set_params(**lgb_params)
model.fit(x_train, y_train,eval_set=[(x_test, y_test)],eval_metric = LoglossObjective())

LGBMClassifier(early_stopping_rounds=500, is_unbalance=True, n_estimators=150,
               num_iterations=5000, objective='binary', random_state=42,
               seed=42, verbose=0)

### random_state

In [33]:
model = _model.random_forest
model.set_params(**random_params)

RandomForestClassifier(max_depth=19, max_leaf_nodes=749, n_estimators=349,
                       random_state=42)

### train_

In [17]:
X = df_train.drop(["is_converted"],axis = 1)
Y = df_train['is_converted']
Y = np.where(Y == False, 0, 1) # catboost

In [52]:
# oversampling
from imblearn.over_sampling import SMOTE, ADASYN
adasyn = SMOTE(sampling_strategy=0.2,random_state=42)
X, Y = adasyn.fit_resample(X, Y)

In [18]:
def score_print(result_dict):
    for name, score_list in result_dict.items():
        print(f'{name} score : {np.mean(score_list):.4f} / STD: (+/- {np.std(score_list):.4f})')

In [19]:
from sklearn.ensemble import GradientBoostingClassifier

In [34]:
gbm_params = {'n_estimators':1000, 'learning_rate' : 1e-3, 'max_depth' : 16, 'random_state' : 42,'verbose':2}

In [35]:
from collections import defaultdict
from sklearn.model_selection import StratifiedKFold
stratkfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_result, valid_result = defaultdict(list), defaultdict(list) # result 리스트
models = [] # 모델을 저장할 리스트

# k-fold
for idx,(train_idx, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_train, x_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    # 모델 선언

    model = GradientBoostingClassifier(**gbm_params)
    # 모델 훈련
    model.fit(x_train, y_train)
    models.append(model) # 훈련된 모델을 리스트에 추가
    train_pred, valid_pred = models[idx].predict(x_train), models[idx].predict(x_test)
    # store result
    for name, score in check_the_score(train_pred, y_train).items(): # train
        train_result[name].append(score)
    for name, score in check_the_score(valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
# print score
print('----[K-Fold Train Score]-----')
score_print(train_result)
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

0it [00:00, ?it/s]

      Iter       Train Loss   Remaining Time 
         1           0.5695            5.07m
         2           0.5676            5.33m
         3           0.5657            5.12m
         4           0.5639            5.01m
         5           0.5620            4.97m
         6           0.5602            4.94m
         7           0.5584            4.94m
         8           0.5567            5.01m
         9           0.5549            4.96m
        10           0.5532            4.81m
        11           0.5515            4.66m
        12           0.5498            4.54m
        13           0.5481            4.42m
        14           0.5464            4.32m
        15           0.5448            4.26m
        16           0.5431            4.18m
        17           0.5415            4.11m
        18           0.5399            4.05m
        19           0.5383            3.99m
        20           0.5367            3.94m
        21           0.5352            3.89m
        2

1it [03:26, 206.58s/it]

      Iter       Train Loss   Remaining Time 
         1           0.5695            3.20m
         2           0.5676            3.41m
         3           0.5657            3.30m
         4           0.5639            3.23m
         5           0.5620            3.21m
         6           0.5602            3.20m
         7           0.5585            3.20m
         8           0.5567            3.25m
         9           0.5549            3.22m
        10           0.5532            3.21m
        11           0.5515            3.21m
        12           0.5498            3.22m
        13           0.5481            3.20m
        14           0.5465            3.19m
        15           0.5448            3.20m
        16           0.5432            3.19m
        17           0.5416            3.19m
        18           0.5400            3.18m
        19           0.5384            3.17m
        20           0.5368            3.17m
        21           0.5352            3.16m
        2

2it [06:51, 205.74s/it]

...

      Iter       Train Loss   Remaining Time 
         1           0.5694            3.41m
         2           0.5675            3.28m
         3           0.5657            3.29m
         4           0.5638            3.28m
         5           0.5620            3.29m
         6           0.5602            3.28m
         7           0.5584            3.25m
         8           0.5566            3.24m
         9           0.5549            3.23m
        10           0.5531            3.22m
        11           0.5514            3.20m
        12           0.5497            3.19m
        13           0.5480            3.19m
        14           0.5463            3.19m
        15           0.5447            3.19m
        16           0.5431            3.18m
        17           0.5414            3.17m
        18           0.5398            3.18m
        19           0.5382            3.18m
        20           0.5366            3.17m
        21           0.5351            3.17m
        2

4it [13:41, 206.12s/it]

      Iter       Train Loss   Remaining Time 
         1           0.5694            3.43m
         2           0.5676            3.28m
         3           0.5657            3.32m
         4           0.5638            3.31m
         5           0.5620            3.32m
         6           0.5602            3.32m
         7           0.5584            3.29m
         8           0.5566            3.27m
         9           0.5549            3.27m
        10           0.5531            3.26m
        11           0.5514            3.24m
        12           0.5497            3.23m
        13           0.5480            3.23m
        14           0.5464            3.23m
        15           0.5447            3.23m
        16           0.5431            3.22m
        17           0.5415            3.21m
        18           0.5399            3.21m
        19           0.5383            3.21m
        20           0.5367            3.20m
        21           0.5351            3.20m
        2

5it [17:12, 206.58s/it]

----[K-Fold Train Score]-----
f1 score : 0.9794 / STD: (+/- 0.0022)
precision score : 0.9601 / STD: (+/- 0.0041)
recall score : 0.9994 / STD: (+/- 0.0003)
roc_auc_score score : 0.9979 / STD: (+/- 0.0003)
----[K-Fold Validation Score]-----
f1 score : 0.7893 / STD: (+/- 0.0066)
precision score : 0.7323 / STD: (+/- 0.0131)
recall score : 0.8565 / STD: (+/- 0.0160)
roc_auc_score score : 0.9163 / STD: (+/- 0.0077)


In [ ]:
----[K-Fold Train Score]-----
f1 score : 0.9883 / STD: (+/- 0.0013)
precision score : 0.9821 / STD: (+/- 0.0021)
recall score : 0.9946 / STD: (+/- 0.0006)
roc_auc_score score : 0.9965 / STD: (+/- 0.0004)
----[K-Fold Validation Score]-----
f1 score : 0.8368 / STD: (+/- 0.0076)
precision score : 0.7820 / STD: (+/- 0.0139)
recall score : 0.9000 / STD: (+/- 0.0069)
roc_auc_score score : 0.9403 / STD: (+/- 0.0033)

In [32]:
# threshold valid
threshold = 0.7
valid_result = defaultdict(list) # result 리스트
for idx, (_, test_idx) in tqdm(enumerate(stratkfold.split(X,Y))):
    x_test, y_test = X.iloc[test_idx], Y[test_idx]

    y_valid_pred = models[idx].predict_proba(x_test)[:,0]
    y_valid_pred = np.where(y_valid_pred >= threshold, False, True)
    
    for name, score in check_the_score(y_valid_pred, y_test).items():  # valid
        valid_result[name].append(score)
        
# print score
print('----[K-Fold Validation Score]-----')
score_print(valid_result)

5it [00:00,  7.02it/s]

----[K-Fold Validation Score]-----
f1 score : 0.8345 / STD: (+/- 0.0045)
precision score : 0.8249 / STD: (+/- 0.0107)
recall score : 0.8446 / STD: (+/- 0.0095)
roc_auc_score score : 0.9144 / STD: (+/- 0.0045)


In [78]:
# feature importance
feature_importances = model.feature_importances_
feature_names = x_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importances))
sorted_importance_dict = {k: v for k, v in sorted(feature_importance_dict.items(), key=lambda item: item[1], reverse=True)}
for feature_name, importance in sorted_importance_dict.items():
    print(f"{feature_name}: {importance}")

lead_owner_converted_ratio: 0.16420310735702515
enterprise: 0.16101905703544617
historical_existing_mean: 0.1049448549747467
com_reg_ver_win_rate: 0.04438501596450806
business_subarea: 0.041253384202718735
business_area: 0.03757695108652115
new_customer_idx: 0.03652491047978401
new_customer_type1: 0.035580508410930634
new_customer_type2: 0.03229067102074623
ver_win_ratio_per_bu: 0.031457651406526566
country: 0.029541203752160072
continent: 0.02902776375412941
ver_win_rate_x: 0.028572633862495422
business_unit: 0.02834302745759487
lead_desc_length: 0.02248220704495907
category_1: 0.02095959708094597
category_3: 0.01845417357981205
customer_interest: 0.01836431212723255
new_expected_timeline: 0.01740826666355133
new_inquiry_type: 0.01719115674495697
job_function: 0.016876664012670517
bant_submit: 0.01686784252524376
category_2: 0.016412343829870224
less_timeline: 0.015508395619690418
seniority_level: 0.01475427858531475


#### inference

In [46]:
test_drop = df_test.drop(['id','is_converted'],axis = 1)

In [50]:
# voting threshold
predicts = []
threshold = 0.9
for model in models:
    pred_proba = model.predict_proba(test_drop)[:,0]
    pred = np.where(pred_proba >= threshold, False, True)
    predicts.append(pred)

In [51]:
for i in predicts:
    print(sum(i))

1056
1063
1044
1031
1034


In [52]:
num_threshold = 1
pred = np.where( np.sum(predicts,axis = 0) >= num_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1688

In [423]:
# difference
pred2 = pred  #pred1 718 | pred2 1142 | pred3 820
print(sum(pred))

1400


In [185]:
su = 0
for i,j in zip(pred3, pred1):
    if i != j:
        su +=1
su

249

## ensemble

In [54]:
df_test['is_converted'] = pred
model_name = 'gbm'
df_test.to_csv(f"{date}_{model_name}.csv",index = False)

In [5]:
date='0223'
# decision_tree: num_threshold: 1 + threshold: 0.9
# xgb: num_threshold: 1, threshold = 0.9
# lgb: num_threshold: 1 + threshold: 0.8
# ramdom_forest: num_threshold: 1 + threshold: 0.9
# cat: num_threshold: 1 + threshold: 0.7
# gbm: num_threshold: 1 + threshold: 0.9
# 1682 1446 1609 1770 1753 1688

In [21]:
# load csv
model_list = ['decision_tree','xgb','lgb','random_forest','cat','gbm','svm']
for idx, model_name in enumerate(model_list):
    df = pd.read_csv(f"{date}_{model_name}.csv")
    globals()[f'pred{idx+1}'] = np.array(df['is_converted'])

In [22]:
for idx in range(len(model_list)):
    print(sum(globals()[f'pred{idx+1}']), end = ' ')

1682 1446 1609 1770 1753 1688 1338 

In [23]:
ensemble_ = np.array([globals()['pred'+ str(idx+1)] for idx in range(len(model_list))])

In [27]:
ensemble_threshold = 4
pred = np.where( np.sum(ensemble_,axis = 0) >= ensemble_threshold, 1, 0)
pred = np.where(pred == 0, False, True)
sum(pred)

1585

## submission

In [28]:
df_test['is_converted'] = pred
df_test.to_csv("submission.csv",index = False)

In [29]:
sum(pred)

1585